In [138]:
import csv

## Laboratório 01 - Parte 1 - Recuperação da informação 2018.1

Nesse lab serão abordados os conceitos de indíce invertido e busca booleana, iremos construir um índice invertido utilizando um dataset com dados de notícias obtidos do jornal online estadão.

##### Leitura dos dados

Primeiramente vamos importar os dados do arquivo csv, processa-los e retorna-los no formato de uma lista de listas onde cada lista contém três campos: titulo, conteudo e o id da noticia.


In [139]:
EMPTY_LIST = []
TITLE = 0
CONTENT = 1
ID = 2

def readData():
    """"Obtains the data from the csv file and returns it as a 
    list of articles, each of them containing title, content and id"""
    data = []
    with open('noticias_estadao.csv', 'rt') as csvFile:
        text = csv.reader(csvFile, delimiter=',')
        lineIdx = 0
        for line in text:
            if lineIdx >= 1:
                data.append(','.join(line).lower().split(','))
            lineIdx += 1
    return data        

##### Construindo o índice

Agora que já temos os dados podemos construir o nosso índice, a estrutura de dados escolhida para o índice foi uma tabela hash onde as chaves da tabela são os termos individuais presentes nas notícias e os valores são listas com os ids das notícias que possuem aquele termo.

Tal estruturação dos dados nos permite realizar consultas lógicas facilmente, podemos consultar por exemplo quais documentos possuem <termo1> E <termo2>, <termo1> OU <termo2> e até mesmo consultas conjutivas que retornam todos os documentos que satisfazem o AND de todos os termos especificados.


In [140]:
def buildInvertedIndex(rawData):
    invertedIndex = {}
    for article in rawData:
        addEntriesToIndex(invertedIndex, article[TITLE], article[ID])
        addEntriesToIndex(invertedIndex, article[CONTENT], article[ID])
    return invertedIndex

##### Adicionando entradas

Temos uma função auxiliar para lidar com a adição de novas entradas no índice. As palavras são separadas e adicionadas no índice.


In [141]:

def addEntriesToIndex(invertedIndex, newsString, articleId):
    """Processes a string extracting the individual words from it 
    and adds <word, list(articleIds)> as a <key, value> pair in the HashMap"""
    for keyword in newsString.split():
        if keyword not in invertedIndex:
            invertedIndex[keyword] = []
        invertedIndex[keyword].append(articleId)


##### Consultas

Agora que já construímos o índice podemos realizar nossas consultas da seguinte forma:

1 - Recuperamos as listas de ids referentes aos termos que queremos buscar<br>
2 - Realizamos a intersecção ou união dessas listas dependendo de qual consulta queremos realizar<br>
3 - Retornamos o resultado<br>


In [142]:

def searchOne(invertedIndex, term):
    """Searches for a specific term in the index and
    returns a list with the relevant article's ids."""
    return list(set(invertedIndex[term]))

def searchAnd(invertedIndex, term1, term2):
    """Searches for documents containing both term1 and term2
    and returns a list with the relevant article's ids."""
    return list(
        set(invertedIndex.get(term1.lower(), EMPTY_LIST)) & 
        set(invertedIndex.get(term2.lower(), EMPTY_LIST))
    )

def searchOr(invertedIndex, term1, term2):
    """Searches for documents containing term1 or term2
    and returns a list with the relevant article's ids."""
    return list(
        set(invertedIndex.get(term1.lower(), EMPTY_LIST)) | 
        set(invertedIndex.get(term2.lower(), EMPTY_LIST))
    )

##### Asserts

Podemos verificar a corretude desta implementação através dos asserts abaixo


In [143]:

def main():
    rawData = readData()
    invertedIndex = buildInvertedIndex(rawData)
    
    #### ASSERTS ####
    
    assert len(searchAnd(invertedIndex, "bolsonaro", "corrupto")) == 0
    assert len(searchAnd(invertedIndex, "bolsonaro", "corrupção")) == 0
    
     #campina, grande (AND)
    assert len(searchAnd(invertedIndex, "campina", "grande")) == 12

    #debate, presidencial (AND e OR)
    assert len(searchOr(invertedIndex, "debate", "presidencial")) == 1770
    assert len(searchAnd(invertedIndex, "debate", "presidencial")) == 201


    #presidenciáveis, corruptos (AND e OR)
    assert len(searchOr(invertedIndex, "presidenciáveis", "corruptos")) == 164
    assert len(searchAnd(invertedIndex, "presidenciáveis", "corruptos")) == 0

    #Belo, Horizonte (AND e OR)
    assert len(searchOr(invertedIndex, "Belo", "Horizonte")) == 331
    assert len(searchAnd(invertedIndex, "Belo", "Horizonte")) == 242
    
    print("Finished. No assertion errors found")  
        
        
if __name__ == "__main__":
    main()

Finished. No assertion errors found
